In [ ]:
# importing necessary packages for data analysis.
import numpy as np
import sys
sys.path.append("../../lhillber/brownian/src")
from time_series import CollectionTDMS as ctdms
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
# data from 07/17 Sagnac/Sagnac_fire_19J is complete shit
# we need to retake this data and make sure that the signal is being 
# taken correctly. For now, I'm using the data from 07/14 because it
# is a better representation of what our data should look like.
d = "../data/20230801/Sagnac/iter_0.tdms"
laser = ctdms(d)
micro = ctdms(d)
laser.set_collection("X")
laser.apply("detrend", mode = "linear", inplace = True)
laser.apply("calibrate", cal = -1, inplace = True)
micro.set_collection("Y")
fig, ax = plt.subplots()

shot = laser.collection[3]
shot.plot(tmin = 4.2e-4, tmax = 4.7e-4, ax = ax)
def friedlander(t, peak, tau):
    return peak * np.exp((-t / tau)) * (1 - (t / tau))

peak_index = np.where(shot.x == max(shot.x))[0][0]
z_crossing = peak_index + np.where(np.diff(np.sign(shot.x[peak_index:])))[0][0]
ax.scatter(shot.t[peak_index], shot.x[peak_index])
ax.scatter(shot.t[z_crossing], shot.x[z_crossing])
guess = np.array([shot.x[peak_index], shot.t[peak_index], shot.t[z_crossing]])
popt, pcov = curve_fit(friedlander, shot.t[peak_index : z_crossing + 500], shot.x[peak_index : z_crossing + 500], maxfev = 100000)
ax.plot(shot.t, friedlander(shot.t, *popt))
print(guess)
print(popt)
ax.set_xlim([4.2e-4, 4.7e-4])
ax.set_ylim([-1, 1.5])
plt.show()

In [10]:
def printSystems(s) -> None:
    for i in s:
        print("\n================================================================================================")
        print("title:\t", i[0])
        print("\nfreq:\t", i[1])
        print("\nphis:\t", i[2])
        print("\nw/o bins:")
        for j in i[3][0]:
            print("\t", j)
        print("\nw/ bins:")
        for j in i[3][1]:
            print("\t", j)
        print("\nsystem freq. cutoff:")
        print("\t", i[4][0])
        print("\nSNR values at cutoff:")
        print("\t", i[4][1])
        print("------------------------------------------------------------------------------------------------\n")
    return None

10


In [1]:
from analysis import System
import numpy as np
names = [("sagnac", "Sagnac", 1e6), ("microphone", "Sagnac", 2e5), ("split beam", "SplitBeam", 1e6), ("telescope", "Telescope", 1e6)]
phis = ["phi" + str(int(np.append(82, np.linspace(145, 160, 16))[i])) + ".tdms" for i in range(len(np.append(82, np.linspace(145, 160, 16))))]
systems = np.array([])
for n in names:
    systems = np.append(systems, System(n[0], ["../data/20230801/" + n[1] + "/iter_0.tdms"] + ["../data/20230801/" + n[1] + "/MinDetect/" + i for i in phis], SNR_freq_cut = n[2], phis = [int(p[3:-5]) for p in phis],  SNR_freq_range = [10000, n[2]]))
for s in systems:
    s.calc_SNR_vs_freq()
# sagnac = System(names[0][0], ["../data/20230801/" + names[0][1] + "/iter_0.tdms"] + ["../data/20230801/" + names[0][1] + "/MinDetect/" + i for i in phis], SNR_freq_cut = names[0][2], phis = [int(p[3:-5]) for p in phis],  SNR_freq_range = [10000, names[0][2]])